In [1]:
%matplotlib inline

In [45]:
import numpy as np
import pandas as pd
import re
import requests

from bs4 import BeautifulSoup as bs

Get death summary page:

In [46]:
url1 = 'https://en.wikipedia.org/w/api.php?action=query&titles=Deaths_in_February_2007&prop=revisions&rvprop=content&format=json'
page1_pages = requests.get(url1).json()['query']['pages']
page1_mkdown = bs(page1_pages[page1_pages.keys()[0]]['revisions'][0]['*'])
page1_txt_list = page1_mkdown.text.split('*')

In [587]:
for i in xrange(0,len(page1_txt_list)):
    first_two_chars = page1_txt_list[i][:2]
    if first_two_chars != '[[':
        print "Error: ", page1_txt_list[i][:40]
        print ""

Error:  {{Deaths in month TOC}}
The following is

Error:  2007-02]]
[[Category:2007-related lists|



In [584]:
# full populated (although some might not have death)
row_pattern_full = re.compile('\[\[(.*?)\]\], (\d+), (?:(.+), )+(.*?). ?\[{1}(h.+)\]{1}')

# missing death and no extra commas
row_pattern_m1 = re.compile('\[\[(.*?)\]\], (\d+), (.+). ?\[{1}(h.+)\]{1}')

#row_pattern_full = re.compile('\[\[(.*?)\]\], (\d+), ((.+), )+(.*?). ?\[{1}(h.+)\]{1}')

count_errors = 0
entry_dict = {}

for entry in page1_txt_list:
    try:
        entry_items = row_pattern_full.match(str(entry.encode('utf-8'))).groups()
        entry_dict[entry_items[0]] = entry_items[1:]
    except AttributeError, e1:
        try:
            entry_items = row_pattern_m1.match(str(entry.encode('utf-8'))).groups()
            entry_dict[entry_items[0]] = list(entry_items[1:3]) + ['', entry_items[3]]
        except AttributeError, e2:
            print "Error: ", entry[:20]
            count_errors += 1

print len(page1_txt_list)
print count_errors

Error:  {{Deaths in month TO
Error:  [[Ismail Semed]], Ch
Error:  [[George André Rober
Error:  2007-02]]
[[Category
235
4


In [575]:
df_entries = pd.DataFrame(entry_dict).T

In [576]:
df_entries.reset_index(inplace=True)
df_entries.columns = ['name', 'age', 'description', 'death', 'url']
#df_entries.columns = ['name', 'age', 'clauses', 'description', 'death', 'url']

In [579]:
df_entries.head()

,name,age,description,death,url
0,Adele Faccio,86,Italian civil right activist.,,http://www.repubblica.it/2007/02/sezioni/polit...
1,Adelina Tattilo,78,Italian founder of ''Playmen'' magazine.,,http://www.repubblica.it/2007/02/sezioni/perso...
2,Ahmad Abu Laban,60,"Egyptian-born Danish Muslim leader, key figure...",cancer,http://www.nytimes.com/2007/02/04/world/europe...
3,Aida Mason,111,British oldest person.,,http://www.24dash.com/communities/16559.htm
4,Alan MacDiarmid,79,New Zealand recipient of [[Nobel Prize in Chem...,injuries from a fall,http://www.nzherald.co.nz/section/1/story.cfm?...


In [580]:
#df_entries[df_entries.name == 'Lew Burdette']['description'][86]
df_entries[df_entries.name == 'Lew Burdette']
df_entries[df_entries.name == 'Fred Mustard Stewart'].values 
df_entries[df_entries.name == 'Aida Mason'].values
df_entries[df_entries.name == 'Ian Richardson'].values

array([['Ian Richardson', '72',
        "British actor (''[[House of Cards (UK TV series)|House of Cards]]'', ''[[Tinker, Tailor, Soldier, Spy]]'') and member of the [[Royal Shakespeare Company|RSC]]",
        'in his sleep',
        'http://news.bbc.co.uk/1/hi/entertainment/6346301.stm] [http://www.rte.ie/arts/2007/0209/richardsoni.html']], dtype=object)